In [1]:
import sys
sys.path.append("../")

import numpy as np 
import pandas as pd
import DataLoader as dl
import tensorflow as tf

from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from IPython.core.display import display, HTML

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

## Load Dataset

In [2]:
loader = dl.Data()
digits = loader.loadData('Emnist', 'Letters')

# Get x and y from data
x = loader.getX(digits)
y = loader.getY(digits)

## Label Encoding

In [3]:
labelencoder = LabelEncoder()
labels = labelencoder.fit_transform(y)
labels

array([19,  5, 18, ..., 20,  3, 18], dtype=int64)

## Split Data

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size = 0.1, random_state = 0)

In [7]:
inputs = keras.Input(shape=(784))

x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(26, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 26)                3354      
Total params: 120,346
Trainable params: 120,346
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

# Train the model for 1 epoch from Numpy data
batch_size = 64
print("Fit on NumPy data")
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1)

Fit on NumPy data
3099/3099 [==============================] - 9s 3ms/step - loss: 0.3675 - acc: 0.8975


In [9]:
predictions = model.predict(x_test)
print(predictions.shape)

(22031, 26)


In [10]:
import time 

preds = []
progress = []

for i in predictions:    
    start = time.time()
    preds.append(np.where(i == np.amax(i))[0][0])
    end = time.time()
    progress.append(end - start)

In [11]:
display(HTML(f'<h2>Accuracy Score: {accuracy_score(y_test, preds)}</h2>'))

In [12]:
mean = np.array(progress)
mean = np.mean(mean)
display(HTML(f'<h2>Prediction Mean time: {mean*1000} ms</h2>'))